In [1]:
import tensorflow as tf
import numpy as np

# [털, 날개] : 없으면 0, 있으면 1
x_data = np.array([
    [0, 0],
    [1, 0],
    [1, 1],
    [0, 0],
    [0, 0],
    [0, 1]
])

# [털, 날개] -> [기타, 포유류 조류]
y_data = np.array([
    [1, 0, 0], # 기타
    [0, 1, 0], # 포유류
    [0, 0, 1], # 조류
    [1, 0, 0], # 기타
    [1, 0, 0], # 기타
    [0, 0, 1]  # 조류
])

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W = tf.Variable(tf.random_uniform([2, 3], -1., 1.))
b = tf.Variable(tf.zeros([3]))

# 연산하기
L = tf.add(tf.matmul(X, W), b)
# 연산 결과를 활성화 함수인 RELU 알고리즘에 대입
L = tf.nn.relu(L)

# softmax 함수는 배열내의 결과값들을 전체 합이 1이 되도록 만들어준다.
model = tf.nn.softmax(L)

# 교차 엔트로피 함수 : 예측값과 실제값 사이의 확률 분포 차이를 계산
# 1) 실제값인 Y 와 예측값인 model의 로그값을 요소끼리 곱한다. < Y * tf.log(model) >
# 2) 1번 결과를 행별로 더해서 배열로 만든다. < reduce_sum(axis=1) >
# 3) 2번 결과의 배열 요소를 평균낸다. < reduce_mean >
# tf.reduce_xxx 함수는 텐서의 차원을 줄여준다. axis매개변수를 이용하여 축소할 차원을 정할 수 있다.
# tf.reduce_sum의 경우 axis=1로 정함으로써 1번째 차원의 값들을 다 더해서 차원을 줄인 것이다.
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))

# 경사하강법 최적화 함수 적용
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

# 텐서플로 세션 초기화 텐서 정의
init = tf.global_variables_initializer()


# 세션 생성 후 실행 시작
sess = tf.Session()
sess.run(init)


# 학습진행 : 100번 반복하기
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    # 학습 도중 10번에 한 번씩 손실값을 출력하기
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

# 학습 결과 확인

# 예측값인 model을 그대로 출력하면 [0.2, 0.7, 0,1]같이 확률로 나오기 때문에
# 요소중 가장 큰 값의 인덱스를 찾아주는 argmax 함수를 이용하여 레이블 값을 출력하도록 함.
prediction = tf.argmax(model, axis=1)
# target은 실제값을 출력하기 위한 것
target = tf.argmax(Y, axis=1)

print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

# 정확도 출력

# 예측값과 실제값을 비교
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict ={X: x_data, Y: y_data}))


10 1.1775
20 1.17573
30 1.17396
40 1.1722
50 1.17046
60 1.16872
70 1.167
80 1.16529
90 1.16359
100 1.1619
예측값: [0 0 0 0 0 0]
실제값: [0 1 2 0 0 2]
정확도: 50.00
